# Performance Test

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mlearn.functional as F
import mlearn as mlearn
import time
import torch
from time import time,sleep
import sys
from ctypes import POINTER, c_double
import numba as nb
np.set_printoptions(4)
torch.set_printoptions(4)
print(np.__version__)

1.17.4


In [6]:
EPOCHS = 10
_inputs = np.random.randn(EPOCHS,32,3,32,32)
_weights = np.random.randn(8,3,5,5)
_bias = np.ones((8))
padding = 3
stride = 2

In [7]:
# 3.15
# 4.29
inputs = mlearn.tensor(_inputs)
weights = mlearn.tensor(_weights)
bias = mlearn.tensor(_bias)
start = time()
for e,batch in enumerate(inputs):
    c_out = F.conv_2d(batch,weights,bias,stride,padding)
#     print(f"\r{e+1}/{EPOCHS}",end="")
#     sys.stdout.flush()
# print()
mlearn_conv = (time() - start)
# ptr = (c_double*24*24*8).from_address(int(c_out[0].data))
# c_out = np.ctypeslib.as_array(ptr)

In [8]:
inputs = torch.tensor(_inputs)

w = torch.tensor(_weights)
b = torch.tensor(_bias)
start = time()
for e,batch in enumerate(inputs):
    out = torch.nn.functional.conv2d(batch,w,b,stride,padding)
#     print(f"\r{e+1}/{EPOCHS}",end="")
#     sys.stdout.flush()
# print()
torch_conv = (time() - start)

In [9]:
print("mlearn -> %.5f"%mlearn_conv)
print("torch  -> %.5f"%torch_conv)
print("比Pytorch慢 %.5f倍"%(mlearn_conv/torch_conv))
np.testing.assert_almost_equal(c_out.data,out.numpy(),5) # check if result is the same

mlearn -> 0.04229
torch  -> 0.00628
比Pytorch慢 6.73066倍


In [43]:
a = np.arange(25).reshape(1,5,5)
stride = 1
padding = 0
print(a.strides)
weight_shape = (1,2,2)

out_x = (5-2)//1+1
out_y = (5-2)//1+1

(200, 40, 8)


In [44]:
a

array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24]]])

In [45]:
arr = np.lib.stride_tricks.as_strided(
    a,
    shape = (1,out_x,out_y,1,2,2),
    strides=a.strides*2
)

In [24]:
arr.shape

(1, 4, 4, 1, 2, 2)

In [25]:
arr[:,0:4:2,0:4:2]

array([[[[[[ 0,  1],
           [ 5,  6]]],


         [[[ 2,  3],
           [ 7,  8]]]],



        [[[[10, 11],
           [15, 16]]],


         [[[12, 13],
           [17, 18]]]]]])